In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,047 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,614 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [21]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

home_data_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_data_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [22]:
# 2. Create a temporary view of the DataFrame.

home_data_df.createOrReplaceTempView('home_sales')

In [29]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price_4bdrm = """
SELECT Year(date_built) AS YEAR,
        AVG(price) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(avg_price_4bdrm).show()


+----+------------------+
|YEAR|     AVERAGE_PRICE|
+----+------------------+
|2017| 296576.6934782609|
|2016|296050.24326347304|
|2015|307908.86020761245|
|2014| 299073.8858447489|
|2013|299999.38822652755|
|2012| 298233.4150805271|
|2011|302141.89581749047|
|2010| 296800.7544776119|
+----+------------------+



In [31]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_3n3 = """
SELECT YEAR(date_built),
        AVG(price) as AVERAGE_PRICE

FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY YEAR(date_built)

"""

spark.sql(avg_price_3n3).show()



+----------------+------------------+
|year(date_built)|     AVERAGE_PRICE|
+----------------+------------------+
|            2015| 288770.2966101695|
|            2013|295962.27145085804|
|            2014| 290852.2661870504|
|            2012| 293683.1872074883|
|            2016|  290555.073964497|
|            2010|  292859.615942029|
|            2011|291117.46706586826|
|            2017| 292676.7887740029|
+----------------+------------------+



In [48]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_3n3n2f = """
SELECT YEAR(date_built),
        AVG(price) as AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and `sqft_living` >= 2000
and floors = 2
GROUP BY YEAR(date_built)


"""

spark.sql(avg_price_3n3n2f).show()


+----------------+------------------+
|year(date_built)|     AVERAGE_PRICE|
+----------------+------------------+
|            2015| 297609.9679144385|
|            2013|      303676.79375|
|            2014| 298264.7183908046|
|            2012|307539.97402597405|
|            2016| 293965.1046511628|
|            2010| 285010.2215909091|
|            2011| 276553.8128654971|
|            2017|280317.57692307694|
+----------------+------------------+



In [50]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = """
SELECT view,
        AVG(price) as AVERAGE_PRICE
FROM home_sales
WHERE price >= 350000
GROUP BY view

"""

spark.sql(view_rating).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     AVERAGE_PRICE|
+----+------------------+
|   7| 403005.7709497207|
|  51| 788128.2083333334|
|  15|404673.29545454547|
|  54| 798684.8235294118|
|  11|399548.11891891895|
|  29| 397771.6512820513|
|  69| 750537.9411764706|
|  42|396964.50241545896|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|   3| 398867.5964912281|
|  30|          397862.0|
|  34| 401419.7451923077|
|  59|          791453.0|
|   8|398592.70658682636|
|  28|402124.62176165805|
|  22| 402022.6847826087|
|  85|1056336.7435897435|
|  16| 399586.5311004785|
+----+------------------+
only showing top 20 rows

--- 1.4847064018249512 seconds ---


In [51]:
# 7. Cache the the temporary table home_sales
spark.sql('cache table home_sales')

DataFrame[]

In [52]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [75]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

cached_data_query = """
SELECT view,
        AVG(price) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000

"""

spark.sql(cached_data_query).show()



print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     AVERAGE_PRICE|
+----+------------------+
|  51| 788128.2083333334|
|  54| 798684.8235294118|
|  69| 750537.9411764706|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|  59|          791453.0|
|  85|1056336.7435897435|
|  52| 733780.2608695652|
|  71| 775651.0952380953|
|  98|1053739.3333333333|
|  99|1061201.4193548388|
|  96|1017815.9166666666|
| 100|         1026669.5|
|  70| 695865.5769230769|
|  61| 746877.5909090909|
|  75|1114042.9428571428|
|  78|1080649.3666666667|
|  89|        1107839.15|
|  77|        1076205.56|
+----+------------------+
only showing top 20 rows

--- 0.8799114227294922 seconds ---


In [57]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_data_df.write.partitionBy('date_built').parquet('p_home_sales_data', mode='overwrite')

In [58]:
# 11. Read the parquet formatted data.
parquet_home_sales_df = spark.read.parquet('p_home_sales_data')

In [59]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales_df.createOrReplaceTempView('p_temp_view')

In [72]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view_rating_query = """
SELECT view,
        AVG(price) as AVERAGE_PRICE
FROM p_temp_view
GROUP BY view
HAVING AVG(price) >= 350000

"""

spark.sql(view_rating_query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     AVERAGE_PRICE|
+----+------------------+
|  51| 788128.2083333334|
|  54| 798684.8235294118|
|  69| 750537.9411764706|
|  73| 752861.1764705882|
|  87|         1072285.2|
|  64| 767036.6666666666|
|  59|          791453.0|
|  85|1056336.7435897435|
|  52| 733780.2608695652|
|  71| 775651.0952380953|
|  98|1053739.3333333333|
|  99|1061201.4193548388|
|  96|1017815.9166666666|
| 100|         1026669.5|
|  70| 695865.5769230769|
|  61| 746877.5909090909|
|  75|1114042.9428571428|
|  78|1080649.3666666667|
|  89|        1107839.15|
|  77|        1076205.56|
+----+------------------+
only showing top 20 rows

--- 1.2563934326171875 seconds ---


In [66]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [67]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False